# Question 1

To see how many players of each rank are playing each level we have we can use the following query (group by Rank instead of selecting)

```sql
SELECT COUNT(*) as num_players
FROM players
JOIN levels ON (players.Level_ID = levels.Level_ID)
GROUP BY Rank, Level_Name;
```

A note on using `COUNT`:
 - Assumes that the players table is distinct, in particular that there is only one record of a player for a given combination of rank, level_id.
 - If assumption does not hold we would have to replace `COUNT` with `COUNT(DISTINCT Player_Id)`.

If we further assume that there is a 1:1 mapping between `Level_ID:LevelName` we could get the result without the join using

```sql
SELECT COUNT(*) as num_players
FROM players
GROUP BY Rank, Level_Id;
```



# Question 2

The linux command downloads using from the second field (tab delimited) for every second row of the first 100 lines in a file.  

However, it iteratively reads a larger and larger chunk of the file to only use the last line of that chunk.

It would be preferable to be interact with the file once by specifying which rows and fields to access.

This can be done in other ways, e.g. using the following one liner:  

```bash
awk 'NR % 2 == 0 && NR <= 100 {print $2}' filename | xargs -n1 wget -c 2> /dev/null
````

Using xargs -n1 ensures that each invocation of wget only takes 1 argument. The invocation is repeatedly executed until standard input is exhausted.

# Question 3

For the purpose of this exercies, it is crucial that each artist and track name can be uniquely identified. 
Deviations would cause the calculations to provide wrong values. If that assumption would not hold, an initial cleaning step would have to be added to the pipeline in this application.

It was found that the artist id and track id columns contained null values, so the artist name and track name columns are used instead. 
The calculations below assume that the data is cleaned and standardized so that each track name and artist has a unique value, e.g. the artist `The Postal Service` and track `Such Great Heights` is not represented in the data in any other shape or form, such as in lower case or with extra spaces etc. 

In [ ]:
spark = SparkSession\
    .builder\
    .config("spark.master", "local")\
    .appName("playstation")\
    .getOrCreate()

### Part A

In [23]:
distinct_songs = spark.read.parquet("../tests/resources/data/output/distinct_songs")

In [18]:
distinct_songs.show()

+-----------+-------------------+
|     userid|distinct_song_count|
+-----------+-------------------+
|user_000066|                666|
|user_000113|               2133|
|user_000098|                254|
|user_000372|               4789|
|user_000424|               2004|
|user_000577|              18227|
|user_000708|               4743|
|user_000289|                994|
|user_000319|               6294|
|user_000445|               3718|
|user_000794|               5743|
|user_000339|               2522|
|user_000821|               1878|
|user_000171|                433|
|user_000182|               9843|
|user_000465|               1440|
|user_000534|               3904|
|user_000706|               4164|
|user_000801|               2761|
|user_000984|               1647|
+-----------+-------------------+
only showing top 20 rows



### Part B

In [21]:
most_popular_songs_df = spark.read.parquet("../tests/resources/data/output/most_popular_songs") 

In [22]:
most_popular_songs_df.show()

+-------------------+--------------------+-----+
|        artist_name|          track_name|count|
+-------------------+--------------------+-----+
| The Postal Service|  Such Great Heights| 3992|
|       Boy Division|Love Will Tear Us...| 3663|
|          Radiohead|        Karma Police| 3534|
|               Muse|Supermassive Blac...| 3483|
|Death Cab For Cutie|     Soul Meets Body| 3479|
|          The Knife|          Heartbeats| 3156|
|               Muse|           Starlight| 3060|
|        Arcade Fire|    Rebellion (Lies)| 3048|
|     Britney Spears|          Gimme More| 3004|
|        The Killers| When You Were Young| 2998|
|           Interpol|                Evil| 2989|
|         Kanye West|       Love Lockdown| 2950|
|     Massive Attack|            Teardrop| 2948|
|Death Cab For Cutie|I Will Follow You...| 2947|
|               Muse| Time Is Running Out| 2945|
|         Bloc Party|             Banquet| 2906|
|        Arcade Fire|Neighborhood #1 (...| 2826|
|          Radiohead

### Part C

In [25]:
top_user_sessions_df = spark.read.parquet("../tests/resources/data/output/top_user_sessions")

In [27]:
top_user_sessions_df.show()

+-----------+-----------------------+---------------------+--------------------+
|     userid|session_start_timestamp|session_end_timestamp|        sorted_songs|
+-----------+-----------------------+---------------------+--------------------+
|user_000949|    2006-02-12 17:49:31|  2006-02-27 11:29:37|[Chained To You, ...|
|user_000997|    2007-04-26 00:36:02|  2007-05-10 17:55:03|[Unentitled State...|
|user_000949|    2007-05-01 02:41:15|  2007-05-14 00:05:52|[White Daisy Pass...|
|user_000544|    2007-02-12 13:03:52|  2007-02-23 00:51:08|[Finally Woken, O...|
|user_000949|    2005-12-09 08:26:38|  2005-12-18 04:40:04|[Neighborhood #2 ...|
|user_000949|    2005-11-11 03:30:37|  2005-11-18 22:50:07|[Excuse Me Miss A...|
|user_000949|    2006-03-18 23:04:14|  2006-03-26 18:13:45|[Disco Science, H...|
|user_000544|    2007-01-06 01:07:04|  2007-01-13 13:57:45|[La Murga, Breath...|
|user_000250|    2008-02-21 15:31:45|  2008-02-28 21:18:03|[Lazarus Heart, S...|
|user_000949|    2006-02-27 